1. Loan tape - fields:user_id	sanction_id	sanction_date	product_type	Product	dpd_as_of_Jan_1_2025	fl_core_policy_user	principal_outstanding	transaction_id	transaction_date	disbursed_amount

2. prod_group	fl_core_policy_user	dpd_bucket	POS

3. Core, experiment,	Term Loan	B2B BNPL	Upskilling	Grand Total - dsibursal, DPD bucket.

In [54]:
import json
import numpy as np
import pandas as pd

In [1]:
import sys
sys.path.append(r'C:/Users/vshnu/OneDrive/Desktop/Vishnuperson/RE/aspire_')
from connector import data_fetch_v2

In [ ]:
#sanction data

billing_1 = f"""
select user_id, dpd, principal_outstanding
from sanction """
flag,sanction_data_df, error_msg = data_fetch_v2(sanction_data, host="13.126.97.63")

In [58]:
sanction_data_df.dtypes

user_id                  object
sanction_id              object
sanction_time    datetime64[ns]
prod_group               object
B2b_partner              object
partner_name             object
dtype: object

Get a B2b partner data from JSON

In [59]:
sanction_data_df["B2b_partner"] = sanction_data_df["json_data"].apply(lambda x: json.loads(x).get("coupon_code_partner",""))

KeyError: 'json_data'

Rename and Remove un necessary columns

In [28]:
sanction_data_df.rename(columns = {'created_time':'sanction_time'},inplace=True)

In [29]:
sanction_data_df.drop(columns = ['json_data'],inplace = True)

In [ ]:
sanction_data_df[sanction_data_df["B2b_partner"]==""]

In [49]:
sanction_data_df["partner_name"] = np.where(((sanction_data_df['B2b_partner'].isna())| (sanction_data_df['B2b_partner']=="")), sanction_data_df['prod_group'], sanction_data_df['B2b_partner'])

In [ ]:
sanction_data_df["partner_name"]# = sanction_data_df['B2b_partner'].fillna(sanction_data_df['prod_group'])


In [63]:
sanction_data_df = sanction_data_df.drop(columns=['prod_group','B2b_partner'])

In [ ]:
#billing data

billing = f"""
select user_id, principal_outstanding, dpd
from billing_info where billing_cycle = 202412 """
flag,billing_df, error_msg = data_fetch_v2(billing, host="13.126.97.63")

In [71]:
type(core_policy_user)

pandas.core.frame.DataFrame

In [88]:
loan_tape_data1 = sanction_data_df.merge(core_policy_user, how = 'left' , on ="user_id")

In [89]:
loan_tape_data2 = loan_tape_data1.merge(billing_df, how = 'left' , on ="user_id")

In [131]:
loan_tape_data = loan_tape_data2.merge(transaction_data_df, how = 'left' , on ="user_id")

In [156]:
loan_tape_data['fl_approved'] = loan_tape_data['fl_approved'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [157]:
loan_tape_data.dtypes

user_id                          object
sanction_id                      object
sanction_time            datetime64[ns]
partner_name                     object
fl_approved                     float64
principal_outstanding           float64
dpd                             float64
transaction_id                   object
created_time             datetime64[ns]
net_disbursed_amount            float64
product                          object
dtype: object

In [132]:
loan_tape_data = loan_tape_data.drop(columns=['Unnamed: 0'])

In [55]:
#core flag
core_policy_user = pd.read_csv(r"C:\Users\vshnu\OneDrive\Desktop\Vishnuperson\RE\aspire_\sql_to_python\core_policy_users.csv")

In [67]:
type(sanction_data_df)

pandas.core.frame.DataFrame

In [ ]:
#txn data

transaction_data = f"""
select user_id, transaction_id, created_time, net_disbursed_amount
from merchant_payments where status = "success" """
flag,transaction_data_df, error_msg = data_fetch_v2(transaction_data, host="13.126.97.63")

In [164]:
loan_tape_data.groupby('dpd')["user_id"].count()

dpd
0.0       30135
239.0         3
263.0         1
270.0        13
293.0         3
          ...  
1241.0       66
1272.0      226
1302.0      163
1333.0      158
1484.0        6
Name: user_id, Length: 89, dtype: int64

In [163]:
loan_tape_data.head(3)

,user_id,sanction_id,sanction_time,partner_name,fl_approved,principal_outstanding,dpd,transaction_id,created_time,net_disbursed_amount,product
0,5828b717-bbd1-46fd-b579-4bc21cd2a0d8,d6be356c-ce5b-4e01-9be3-f4ddff164e31,2020-11-12 04:10:14,4M,1.0,NaN,NaN,1d68a2f0-f7b1-4143-a244-277bb394f171,2021-01-04 20:59:38,351.0,Core product
1,5828b717-bbd1-46fd-b579-4bc21cd2a0d8,d6be356c-ce5b-4e01-9be3-f4ddff164e31,2020-11-12 04:10:14,4M,1.0,NaN,NaN,b8d69a72-0217-4004-aa93-753b165e6ae3,2021-01-12 17:38:40,100.0,Core product
2,5828b717-bbd1-46fd-b579-4bc21cd2a0d8,d6be356c-ce5b-4e01-9be3-f4ddff164e31,2020-11-12 04:10:14,4M,1.0,NaN,NaN,0493daf9-fca9-41e8-979e-3e161f6a17e5,2021-01-16 22:25:10,20.0,Core product


In [169]:
loan_tape_data["dpd"] = loan_tape_data["dpd"].astype(str)

In [198]:
loan_tape_data.dtypes

user_id                          object
sanction_id                      object
sanction_time            datetime64[ns]
partner_name                     object
fl_approved                     float64
principal_outstanding           float64
dpd                               int64
transaction_id                   object
created_time                     object
net_disbursed_amount            float64
product                          object
dpd_bucket                       object
dtype: object

In [183]:
loan_tape_data['dpd'] = loan_tape_data['dpd'].replace('nan','0.0')

In [193]:
loan_tape_data.groupby("dpd")['user_id'].count()

dpd
0       290712
239          3
263          1
270         13
293          3
         ...  
1241        66
1272       226
1302       163
1333       158
1484         6
Name: user_id, Length: 89, dtype: int64

In [192]:
loan_tape_data["dpd"] = loan_tape_data["dpd"].apply(lambda x: int(x))

In [195]:
loan_tape_data["dpd_bucket"] = loan_tape_data["dpd"].apply(lambda x: 'A:current' if x == 0 else
                                                           'B:1_360' if x>0 and x<=360 else
                                                           'C:361_720' if x>360 and x<=720
                                                           else 'D:>720')

In [197]:
loan_tape_data.head(2)

,user_id,sanction_id,sanction_time,partner_name,fl_approved,principal_outstanding,dpd,transaction_id,created_time,net_disbursed_amount,product,dpd_bucket
0,5828b717-bbd1-46fd-b579-4bc21cd2a0d8,d6be356c-ce5b-4e01-9be3-f4ddff164e31,2020-11-12 04:10:14,4M,1.0,0.0,0,1d68a2f0-f7b1-4143-a244-277bb394f171,2021-01-04 20:59:38,351.0,Core product,A:current
1,5828b717-bbd1-46fd-b579-4bc21cd2a0d8,d6be356c-ce5b-4e01-9be3-f4ddff164e31,2020-11-12 04:10:14,4M,1.0,0.0,0,b8d69a72-0217-4004-aa93-753b165e6ae3,2021-01-12 17:38:40,100.0,Core product,A:current


In [196]:
loan_tape_data.groupby('dpd_bucket')['user_id'].count()


dpd_bucket
A:current    290712
B:1_360       18551
C:361_720     48538
D:>720        27177
Name: user_id, dtype: int64

In [ ]:
import pandas as pd

# Example DataFrame


# Define product categorization logic
def categorize_product(row):
    x, y = row['partner_name'], row['fl_approved']
    
    if x in ['4M'] and y in [1.0]:
        return 'Core product'
    elif x in ['4M'] and y not in [1.0]:
        return "experiment product"
    elif x in ['FLPKRT_1M', 'APNA', 'NINJA_1M', 'REDB_1M', 'BZTM_1M', 'yodacart', 'roinet', 
               'APNA_4M', 'BIDSO_1M', "gramophone", 'essamrt', 'APNA_5M', 'ANX_1M', 
               'EKAR_1M', 'BZ_4M', 'TRVCLN', 'buyume']:
        return 'B2B'
    elif x in ['PL']:
        return 'Term loans'
    else:
        return 'Others'

# Apply function row-wise
loan_tape_data["product"] = loan_tape_data.apply(categorize_product, axis=1)

# Display DataFrame
print(loan_tape_data)


In [ ]:

# Get the rows where index is minimum for each user_id
loan_tape_pos = loan_tape_data.loc[loan_tape_data.groupby('user_id').apply(lambda x: x.index.min())]

# Reset index if needed
loan_tape_pos.reset_index(drop=True, inplace=True)

# Display the new DataFrame
print(loan_tape_pos)


In [202]:
loan_tape_pos.head(2)

,user_id,sanction_id,sanction_time,partner_name,fl_approved,principal_outstanding,dpd,transaction_id,created_time,net_disbursed_amount,product,dpd_bucket
0,00074193-df73-41d2-b1e1-ce7d6b9d015c,2752e794-b2e4-45e2-8dbb-4f01d2b0463c,2022-04-15 11:52:11,4M,0.0,0.00,0,0.0,0.0,0.0,experiment product,A:current
1,000ade4b-897e-450a-89f2-2f5e3078737e,8d9f62d3-863f-4e60-907a-b9c13c40dae2,2023-01-27 11:56:09,4M,0.0,6239.46,480,b6d277ef-a94d-4f4e-a1be-d26979308974,2023-04-03 21:33:06,36.0,experiment product,C:361_720


In [211]:
# loan_tape_pos.groupby(["product","dpd_bucket"])['principal_outstanding'].sum()
pivot_df = loan_tape_pos.groupby(["product","fl_approved","dpd_bucket"])['principal_outstanding'].sum().unstack(fill_value=0)

pivot_df

dpd_bucket                      A:current    B:1_360   C:361_720      D:>720
product            fl_approved                                              
B2B                0.0               0.00    5500.00   104712.65        0.00
                   1.0               0.00  151100.00   584958.87     7999.00
Core product       1.0               0.85  918382.18  7295523.83  4003384.20
Others             0.0               0.00   11500.00        0.00      630.75
Term loans         0.0               0.00   35333.35   946279.13        0.00
                   1.0               0.67  594186.24  6941453.63        0.00
experiment product 0.0               0.54  650935.86  4022383.90  5923231.91

In [220]:
pivot_disb = loan_tape_data.groupby(['product', 'disb_quarter'])['net_disbursed_amount'].sum().unstack(fill_value=0)
pivot_disb

disb_quarter,1970Q1,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q2,2024Q3,2024Q4,2025Q1
product,,,,,,,,,,,,,,,,,,
B2B,0.0,0.0,0.0,0.0,0.0,0.00,0.00,167680.26,1332733.16,5437091.49,8681819.00,8250265.87,13109860.18,10909202.25,6867.34,141984.28,127093.82,172795.73
Core product,0.0,27826.0,237471.0,1741997.0,1991501.0,3583940.01,16527645.80,43366097.80,41096128.91,46652990.43,54607932.28,64294481.17,60037474.83,31383061.66,0.00,0.00,0.00,0.00
Others,0.0,0.0,0.0,0.0,0.0,0.00,268395.28,312531.65,243025.00,215679.85,264647.00,240361.16,231765.00,160977.00,0.00,1794600.00,9673355.00,2941500.00
Term loans,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,5230796.50,43659796.38,2957701.82,0.00,0.00,0.00,0.00
experiment product,0.0,0.0,37489.0,910290.0,1253166.0,2895815.00,13393164.27,24579702.71,22034011.91,20484787.37,18710522.77,21060155.96,18277442.43,9769502.26,0.00,0.00,0.00,0.00


In [215]:
loan_tape_data["created_time"] = pd.to_datetime(loan_tape_data["created_time"])

In [216]:
loan_tape_data["disb_quarter"] = loan_tape_data["created_time"].dt.to_period('Q')